In [1]:
# site url 가져오기
# google 에서 texas Cotton Association directory 검색 후 Association directory 접근
url = 'https://nasaaorganic.org.au/find-organic-producers-products/product-search/'

In [2]:
from selenium import webdriver

# 웹드라이버 설정 (예: Chrome의 경우)
driver = webdriver.Chrome()

In [15]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

driver.get(url)
wait = WebDriverWait(driver, 10)
search_button = wait.until(EC.element_to_be_clickable((By.ID, 'Search')))

search = driver.find_element(By.ID, "Search")
search.click()

detail_list = driver.find_elements(By.CSS_SELECTOR, ".btn.btn-light.contactDetailsBtn")


search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.btn-light.contactDetailsBtn")))

html_content = driver.page_source
soup = BeautifulSoup(html_content, 'html.parser')

    

In [79]:
first = soup.find('h4')
loop = first.find_next('h4')
print(first)
print(loop)

name = next(first.descendants)
first.next_sibling.next_sibling

prev = first
details = {}
for i in range(3):
    strong_tag = prev.find_next('strong')
    key = strong_tag.get_text(strip=True).rstrip(':')
    value = strong_tag.next_sibling
    if value:
        details[key] = value.strip()

content = prev.find_next('div', {'class': 'contactDetails'})
content.parent.previous_sibling.previous_sibling.previous_sibling



<h4> N.T. Cattlemens Trading Pty Ltd <small>(Certification No. 0028P)</small></h4>
<h4>5th Acre <small>(Certification No. 4235)</small></h4>


'Hugh  NT AUSTRALIA '

In [87]:
# first = soup.find('h4')
# loop = first.find_next('h4')
# print(first)
# print(loop)

# name = next(first.descendants)
# first.next_sibling.next_sibling


# Extracting the h4 tag content

def get_details(name_tag):
    prev = name_tag
    details = {}
    for i in range(3):
        strong_tag = prev.find_next('strong')
        key = strong_tag.get_text(strip=True).rstrip(':')
        value = strong_tag.next_sibling

        if key not in {'Certification Program', 'Category', 'Certified For'}:
            break
        
        if value:
            details[key] = value.strip()

        prev = strong_tag

    content = prev.find_next('div', {'class': 'contactDetails'})
    details['address'] = content.parent.previous_sibling.previous_sibling.previous_sibling.text.strip()

    # Extracting contact details
    contact_div = prev.find_next('div', {'class': 'contactDetails'})
    if contact_div:
        for strong_tag in contact_div.find_all('strong'):
            key = strong_tag.get_text(strip=True).rstrip(':')
            value = strong_tag.next_sibling
            if value:
                details[key] = value.strip()
    return details


result_list = []
h4_content = None
while(True):
    if not h4_content:
        h4_content = soup.find('h4')
    else:
        h4_content = h4_content.find_next('h4')

    if not h4_content:
        break
    
    try:
        result = {"name": next(h4_content.descendants).text.strip()}
        detail = get_details(h4_content)
        result.update(detail)

        result_list.append(result)
    except Exception as e:
        print(e)
        break

'NoneType' object has no attribute 'get_text'


In [92]:
result_list[0]

{'name': 'N.T. Cattlemens Trading Pty Ltd',
 'Certification Program': 'DoA Processor/Handler (Organic); NOP Processor/Handler (Organic)',
 'Category': 'Services',
 'Certified For': 'Spelling Yard (Handling of Certified Organic Livestock and Hay)',
 'address': 'Hugh  NT AUSTRALIA'}

In [93]:
import pandas as pd
df = pd.DataFrame(result_list)
df.to_csv("./output/nasaa_organic_20240201.csv", quoting=1, index=False)

In [94]:
result_list[-1]

{'name': 'ZERO Slim & Healthy',
 'Certification Program': 'DoA Processor/Handler (Certified; Organic)',
 'Category': 'Seeds and Cereal Products, including Grains, Roots and Tubers, Pulses, Legumes, Sago and Cassava; Services',
 'Certified For': 'Exporting (Exporting of ZERO Slim & Healthy Noodles, Rice & Pasta products); Pastas, Noodles and Similar Products (ZERO Slim & Healthy Angel Hair Noodles, ZERO Slim & Healthy Fettucine, ZERO Slim & Healthy Lasagna, ZERO Slim & Healthy Rice)',
 'address': 'Pooraka  SA AUSTRALIA',
 'Mobile': '0400 998 766',
 'Email': 'info@slimandhealthysecrets.com.au'}

In [30]:
first

<h4> N.T. Cattlemens Trading Pty Ltd <small>(Certification No. 0028P)</small></h4>

In [ ]:
# 웹드라이버 종료
driver.quit()